# MQTT server

In [1]:
import paho.mqtt.client as mqtt
from paho.mqtt.properties import Properties
from paho.mqtt.packettypes import PacketTypes
from joserfc import jws
import pprint
import time
import json
import base64
from datetime import datetime as dt
from datetime import timezone as tz
from joserfc.jwk import ECKey

# Make query type readable
qtype = { 1 : 'A', 2 : 'NS', 3 : 'MD (obsolete)', 4 : 'MF (obsolete)', 5 : 'CNAME',
          6 : 'SOA', 7 : 'MB (exp)', 8 : 'MG (exp)', 9 : 'MR (exp)', 10 : 'NULL (exp)',
         11 : 'WKS', 12 : 'PTR', 13 : 'HINFO', 14 : 'MINFO', 15 : 'MX',
         16 : 'TXT', 17 : 'RP', 18 : 'AFSDB', 19 : 'X25', 20 : 'ISDN',
         21 : 'RT', 22 : 'NSAP (deprecated)', 23 : 'NSAP-PTR (deprecated)', 24 : 'SIG',
         25 : 'KEY',26 : 'PX', 27 : 'GPOS', 28 : 'AAAA', 29 : 'LOC', 
         30 : 'NXT (obsolete)', 31 : 'EID', 32 : 'NIMLOC', 33 : 'SRV', 34 : 'ATMA',
         35 : 'NAPTR', 36 : 'KX', 37 : 'CERT', 38 : 'A6 (obsolete)', 39 : 'DNAME',
         40 : 'SINK', 41 : 'OPT', 42 : 'APL', 43 : 'DS', 44 : 'SSHFP', 45 : 'IPSECKEY',
         46 : 'RRSIG', 47 : 'NSEC', 48 : 'DNSKEY', 49 : 'DHCID', 50 : 'NSEC3',          
         51 : 'NSEC3PARAM', 52 : 'TLSA', 53 : 'SMIMEA', 55 : 'HIP',
         56 : 'NINFO', 57 : 'RKEY', 58 : 'TALINK', 59 : 'CDS', 60 : 'CDNSKEY',
         61 : 'OPENPGPKEY', 62 : 'CSYNC', 63 : 'ZONEMD', 64 : 'SVCB', 65 : 'HTTPS',

         99 : 'SPF', 100 : 'UINFO', 101 : 'UID', 102 : 'GID', 103 : 'UNSPEC',
        104 : 'NID', 105 : 'L32', 106 : 'L64', 107 : 'LP', 108 : 'EUI48',
        109 : 'EUI64',

        249 : 'TKEY', 250 : 'TSIG', 251 : 'IXFR', 252 : 'AXFR', 253 : 'MAILB (exp)',
        254 : 'MAILA (obsolete)', 255 : 'ANY (*)', 256 : 'URI', 257 : 'CAA',
        258 : 'AVC', 259 : 'DOA', 260 : 'AMTRELAY', 261 : 'RESINFO',

        32768 : 'TA', 32769 : 'DLV (obsolete)'      
       }

# Make response code readable
rcode = ['NOERROR', 'FORMERR', 'SERVFAIL', 'NXDOMAIN', 'NotImp', 'REFUSED', 'XYDOMAIN', 
         'XYRRSet', 'NXRRSet', 'NotAuth', 'NotZone', '', '', '', '', '']

# MQTT topics used
eventsdn = 'events/down/pop/general'
eventsup = 'events/up/#'

# MQTT broker address
broker = 'mqtt.example.com'

# Signing certificate files
certfile = ".tls/signer.pem"
signkey = ".tls/signer-key.pem"


## Create Observation packet function

In [2]:
# Send Observation to Edge Policy Manager (POP)
def observation_packet(client, domain, tag):
    # Generate test packet
    # Create an mqtt message with a single domain
    # with all relevant tag bits set

    ts = dt.now(tz=tz.utc).isoformat()

    # Domain data
    #  Name: the domain name to act on
    #  TimeAdded: when the observation was made
    #  TTL: how long POP should act on this information
    #  TagMask: aggregated attributes related to the domain namn
    #
    dom = {
       'Name' : domain,
       'TimeAdded': ts, 
       'TTL': 1200, 
       'TagMask': tag
      }

    # Domain data that is added to POP
    added = [ dom ]

    # Domain data that is deleted from POP
    deleted = []

    # MQTT message
    msg = {
       'SrcName': 'TAPIR Core',
       'MsgType': 'observation',
       'ListType': 'greylist',
       'Added': added,
       'Deleted' : deleted,
       'Msg': '',
       'TimeStamp': ts,
       'Creator': 'core.example.com.'
      }

    # Message signature
    # Currently EC/P-256, should be signed with Core signature
    # POP only trusts data with proper signature
    member = {
           'protected': {'alg': 'ES256'},
         }

    with open(signkey) as keyf:
        privatekey = ECKey.import_key(keyf.read())

    payload = json.dumps(msg)
    out = json.dumps(jws.serialize_json(member, payload, privatekey))

    # Send a single MQTT event without creating a persistent client
    # publish.single(eventsdn, out, qos=2, hostname=broker, protocol=mqtt.MQTTv5)

    # Use existing client
    properties=Properties(PacketTypes.PUBLISH)
    properties.MessageExpiryInterval=30 # in seconds
    client.publish(eventsdn, out, 2, properties=properties)


## Create MQTT callbacks

In [3]:
# Standard Paho Python library example callbacks, except where noted

# Connect callback
def on_connect(client, userdata, flags, reason_code, properties):
    ts = dt.now().strftime("%H:%M:%S.%f")[:-2]
    print(f"{ts} Connecting: {reason_code}")

# Disconnect callback
def on_disconnect(client, userdata, flags, reason_code, properties):
    ts = dt.now().strftime("%H:%M:%S.%f")[:-2]
    print(f"{ts} DisConnecting: {reason_code}")

# Message callback
# Incudes a mechanism where all incoming domains (New domain from Edge)
# that have the suffix "example.com" get sent as an Observation event
# to Edge Policy Processor (POP)
# Example: 
#   ho.ho.ho.im.a.domain.name.example.com.
# results in an observation packet being sent to POP
# for the domain "ho.ho.ho.im.a.domain.name."
#
def on_message(client, userdata, message):
    ts = dt.now().strftime("%H:%M:%S.%f")[:-2]
    creator = message.topic.split('/')[2]
    dta = json.loads(base64.urlsafe_b64decode(json.loads(message.payload)['payload'] + "=="))
    # In example.com is seen on incoming data
    if 'qname' in dta:
        if dta["qname"][-12:] == "example.com.":
            # Trunkate and send observation
            dmn=dta["qname"][:-12]
            observation_packet(client, dmn, 96)
            print(f" == Sent observation for {dmn}")
        print(dta["qname"], qtype[dta["qtype"]], rcode[dta["flags"] & 15], dta["timestamp"])

# Published message callback
def on_publish(client, userdata, mid, reason_code, properties=None):
    ts = dt.now().strftime("%H:%M:%S.%f")[:-2]
    print(f"{ts} Published message id: {str(mid)}")

# Subscribe callback
def on_subscribe(client, userdata, mid, reason_code, properties=None):
    ts = dt.now().strftime("%H:%M:%S.%f")[:-2]
    print(f"{ts} Subscribe: {reason_code}")

# Unsubscribe callback
def on_unsubscribe(client, userdata, mid, reason_code, properties=None):
    ts = dt.now().strftime("%H:%M:%S.%f")[:-2]
    print(f"{ts} Unsubscribe: {reason_code}")

# Logging callback
def on_log(client, userdata, paho_log_level, messages):
    if paho_log_level == mqtt.LogLevel.logging.DEBUG:
        print(f"Log: {messages}")


## Main

In [ ]:
if __name__ == '__main__':

    client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2,
                         # client_id="sparky",
                         transport='tcp',
                         userdata = {},
                         protocol=mqtt.MQTTv5)

    # Check signature file
    with open(certfile) as keyf:
        pubkey = ECKey.import_key(keyf.read())
    client.user_data_set({"pubkey": pubkey})

    # Callbacks
    
    # Connect
    client.on_connect = on_connect
    client.on_disconnect = on_disconnect

    # Subscribe
    client.on_subscribe = on_subscribe
    client.on_unsubscribe = on_unsubscribe

    # Messages
    client.on_message = on_message
    client.on_publish = on_publish

    # Log
    client.on_log = on_log

    properties=Properties(PacketTypes.CONNECT)
    properties.SessionExpiryInterval=30*60

    client.connect(broker,
               port=1883,
               clean_start=mqtt.MQTT_CLEAN_START_FIRST_ONLY,
               properties=properties,
               keepalive=60);

    client.loop_start()

    time.sleep(2)
    client.subscribe(eventsup,2)
    # client.subscribe(eventsdn,2)

    ts = dt.now().isoformat(" ", timespec="seconds")
    while True:    
        if ts < dt.now().isoformat(" ", timespec="hours"):
            print(f"{ts} still running")
        ts = dt.now().isoformat(" ", timespec="seconds")
        time.sleep(60)

    client.unsubscribe(eventsdn)
    client.disconnect()
    client.loop_stop()